# 4_ReviseCareAI_AdvisorySystem

## Purpose of that notebook - to give details report per hospital using gathered and processed with GenAI data

In [1]:
%run 0_ReviseCareAI_Utils

StatementMeta(, 96aba115-d73b-43b4-a444-23cc4857c4c0, 13, Finished, Available, Finished)

Variables GMAP_API_KEY, OPENAI_API_KEY, AZURE_OPEN_AI_KEY, AZURE_OPEN_AI_ENDPOINT were initialized
Google client gmaps was is initialized
Function list_to_dataframe() is initialized
Function get_ids_from_config() is initialized
Function get_hospital_details() is initialized
Function openai_get_answer() is initialized
UDF Function openai_udf() is initialized
Function string_masking() is initialized
UDF Function string_masking_udf() is initialized


# Recommendation System

In [2]:
enriched_genai = spark.read.format("delta").load("Files/enriched_genai")
# display(enriched_genai)
enriched_genai.cache()
enriched_genai.createOrReplaceTempView("genai_view")

StatementMeta(, 96aba115-d73b-43b4-a444-23cc4857c4c0, 14, Finished, Available, Finished)

In [3]:
%%sql
CREATE OR REPLACE TEMP VIEW suggestions_view AS
with cte1 AS (
    SELECT 
        row_number() OVER (PARTITION BY hospital_name ORDER BY review_time) as review_number
        ,hospital_name
        ,review_text
        ,review_rating
        ,department
        ,review_time
    FROM 
        genai_view
    WHERE
        1=1
)

,cte2 AS (
    SELECT
        hospital_name
        ,CONCAT('Review', review_number, ': ', review_text, ' ', '\nDepartment: ', department, '\nReview time: ', review_time, ' \n') as reviews
    FROM cte1
)

,cte3 AS (
SELECT
    hospital_name
    ,concat_ws(',', collect_list(reviews)) AS gen_ai_context 
FROM cte2
GROUP BY hospital_name
)

SELECT
    *
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals." 
        ,"Give concise and structured answer with 3-5 points which problems were detected in that hospital"
        ,gen_ai_context) as hospital_challenges
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals." 
        ,"Give concise and structured answer with 3-5 points where the hospital doing good"
        ,gen_ai_context) as hospital_strengths
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals."
        ,"""Analyze patient reviews to identify whether there are problems ONLY regarding flexible scheduling. Ignore other problems.
        Your answer should first raise the overall opinion: Good/Bad/No mentions. After, some particular details"""
        ,gen_ai_context) as hospital_scheduling
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals."
        ,"""Analyze patient reviews to identify whether there are problems ONLY regarding telehealth. Ignore other problems.
        Your answer should first raise the overall opinion: Good/Bad/No mentions. After, some particular details"""
        ,gen_ai_context) as hospital_telehealth
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals."
        ,"""Analyze patient reviews to identify whether there are problems ONLY regarding senior care. Ignore other problems.
        Your answer should first raise the overall opinion: Good/Bad/No mentions. After, some particular details"""
        ,gen_ai_context) as hospital_seniorcare
    ,openai_udf(
        "Below, you will see the recent reviews on Google Map for Hospitals."
        ,"""Analyze patient reviews to identify whether there are problems ONLY regarding wellness services. Ignore other problems.
        Your answer should first raise the overall opinion: Good/Bad/No mentions. After, some particular details"""
        ,gen_ai_context) as hospital_wellness
FROM cte3

StatementMeta(, 96aba115-d73b-43b4-a444-23cc4857c4c0, 15, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [4]:
suggestions_df = spark.sql("select * from suggestions_view")
suggestions_df.cache()

StatementMeta(, 96aba115-d73b-43b4-a444-23cc4857c4c0, 16, Finished, Available, Finished)

DataFrame[hospital_name: string, gen_ai_context: string, hospital_challenges: string, hospital_strengths: string, hospital_scheduling: string, hospital_telehealth: string, hospital_seniorcare: string, hospital_wellness: string]

In [5]:
def buildReport(hospital='Sharp Grossmont Hospital', df = suggestions_df):
    df = df.filter(df['hospital_name']==hospital)
    
    print(f"REPORT FOR THE HOSPITAL - {hospital}")

    row = df.first()
    row_dict = row.asDict()
    
    
    print('\n********* HOSPITAL CHALLENGES *********')
    print(row_dict['hospital_challenges'])
    

    # hospital_strengths
    print('\n********* HOSPITAL STRENGTHS *********')
    print(row_dict['hospital_strengths'])
    
    # hospital_scheduling
    print('\n********* ANALYZING HOSPITAL SCHEDULING ISSUES *********')
    print(row_dict['hospital_scheduling'])


    # hospital_telehealth
    print('\n********* ANALYZING HOSPITAL TELEHEALTH ISSUES *********')
    print(row_dict['hospital_telehealth'])


    # hospital_seniorcare
    print('\n********* ANALYZING HOSPITAL SENIORCARE ISSUES *********')
    print(row_dict['hospital_seniorcare'])

    # hospital_wellness
    print('\n********* ANALYZING HOSPITAL WELLNESS ISSUES *********')
    print(row_dict['hospital_wellness'])

StatementMeta(, 96aba115-d73b-43b4-a444-23cc4857c4c0, 17, Finished, Available, Finished)

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def getListValuesFromDF(df, col):
    return df.select(col).distinct().orderBy(col).rdd.flatMap(lambda x: x).collect()

hospital_dropdown = widgets.Dropdown(
    options=getListValuesFromDF(df=suggestions_df, col='hospital_name'),
    value=getListValuesFromDF(df=enriched_genai, col='hospital_name')[0],
    description='Hospital Name:',
)

submit_button = widgets.Button(
    description='Show Report',
    button_style='',
    tooltip='Click to submit',
    icon='check'
)

def on_submit_button_clicked(b):
    clear_output(wait=True)
    display(hospital_dropdown, submit_button)
    
    buildReport(hospital = hospital_dropdown.value)

submit_button.on_click(on_submit_button_clicked)
display(hospital_dropdown, submit_button)

Dropdown(description='Hospital Name:', index=2, options=('Masked:Alqlus Akluxdqsr Jqrkxsdq - Adj Gxlwq', 'Mask…

Button(description='Show Report', icon='check', style=ButtonStyle(), tooltip='Click to submit')

REPORT FOR THE HOSPITAL - Masked:Aunxkkr Ilnur Jqrkxsdq Fznqd Hxrsd

********* HOSPITAL CHALLENGES *********
Based on the reviews provided, the following problems were detected in the hospital:

1. **Outdated Facilities**: One reviewer mentioned that the hospital is not very updated, indicating a need for renovations or modern amenities.

2. **Inconsistent Quality of Care**: While many reviews praised the staff for their kindness and helpfulness, one review highlighted a negative experience in the emergency department where a patient felt belittled and mistreated by a nurse.

3. **Staff Attitude Issues**: A specific incident was reported where a doctor exhibited an unprofessional attitude towards a patient, suggesting that staff behavior can vary significantly and may impact patient care.

4. **Concerns in Emergency Department**: One review pointed out a poor experience in the emergency room regarding pain assessment and the perceived lack of empathy from certain staff members.

5. **L